#### 在许多应用中，数据可能分散在许多文件或数据库中，存储的形式也不利于分析。本章关注可以聚合、合并、重塑数据的方法。首先，我会介绍 pandas 的层次化索引，它广泛用于以上操作。然后，我深入介 绍了一些特殊的数据操作。在第 14 章，你可以看到这些工具的多种应用。


#### 层次化索引(hierarchical indexing)是 pandas 的一项重要功能，它使你能 在一个轴上拥有多个(两个以上)索引级别。抽象点说，它使你能以低维度形 式处理高维度数据。我们先来看一个简单的例子:创建一个 Series，并用一个 由列表或数组组成的列表作为索引:


In [4]:
import pandas as pd
import numpy as np
data = pd.Series(np.random.randn(9),index=[['a','a','a','b','b','c','c','d','d'],[1,2,3,1,3,1,2,2,3]])
data

a  1    0.995173
   2   -0.102927
   3   -0.943508
b  1    0.875500
   3    0.871755
c  1    0.273639
   2    2.773223
d  2   -1.036911
   3    1.230428
dtype: float64

#### 看到的结果是经过美化的带有 MultiIndex 索引的 Series 的格式。索引之间的 “间隔”表示“直接使用上面的标签”:


In [5]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

#### 对于一个层次化索引的对象，可以使用所谓的部分索引，使用它选取数据子集的操作更简单:


In [6]:
data['b']

1    0.875500
3    0.871755
dtype: float64

In [8]:
data['b':'c']

b  1    0.875500
   3    0.871755
c  1    0.273639
   2    2.773223
dtype: float64

In [9]:
data.loc[['b','d']]

b  1    0.875500
   3    0.871755
d  2   -1.036911
   3    1.230428
dtype: float64

#### 有时甚至还可以在“内层”中进行选取:

In [10]:
data.loc[:,2]

a   -0.102927
c    2.773223
d   -1.036911
dtype: float64

#### 层次化索引在数据重塑和基于分组的操作(如透视表生成)中扮演着重要的角 色。例如，可以通过 unstack 方法将这段数据重新安排到一个 DataFrame 中:


In [11]:
data.unstack()

,1,2,3
a,0.995173,-0.102927,-0.943508
b,0.875500,NaN,0.871755
c,0.273639,2.773223,NaN
d,NaN,-1.036911,1.230428


#### 在用pandas进行数据重排时，经常用到stack和unstack两个函数。stack的意思是堆叠，堆积，unstack即“不要堆叠”，我对两个函数是这样理解和区分的。

#### 表格在行列方向上均有索引（类似于DataFrame），花括号结构只有“列方向”上的索引（类似于层次化的Series），结构更加偏向于堆叠（Series-stack，方便记忆）。stack函数会将数据从”表格结构“变成”花括号结构“，即将其行索引变成列索引，反之，unstack函数将数据从”花括号结构“变成”表格结构“，即要将其中一层的列索引变成行索引。

#### 对于一个 DataFrame，每条轴都可以有分层索引:

In [13]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),index = [['a','a','b','b'],[1,2,1,2]],columns=[['Ohio','Ohio','Colorado'],
                                                                                                 ['Green','Red','Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

#### 各层都可以有名字(可以是字符串，也可以是别的 Python 对象)。如果指定了 名称，它们就会显示在控制台输出中:


In [14]:
frame.index.names = ['key1','key2']
frame.columns.names = ['state','color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

#### 注意:小心区分索引名 state、color 与行标签。

#### 有了部分列索引，因此可以轻松选取列分组:

In [15]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

#### 可以单独创建 MultiIndex 然后复用。上面那个 DataFrame 中的(带有分级名 称)列可以这样创建:


### 重排与分级排序

#### 有时，你需要重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据 进行排序。swaplevel 接受两个级别编号或名称，并返回一个互换了级别的新 对象(但数据不会发生变化):


In [17]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

#### 而 sort_index 则根据单个级别中的值对数据进行排序。交换级别时，常常也会 用到 sort_index，这样最终结果就是按照指定顺序进行字母排序了:


In [18]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

#### 作用：默认根据行标签对所有行排序，或根据列标签对所有列排序，或根据指定某列或某几列对行排序。